# Programación Dinámica

Vamos a ver una aplicación más compleja y con muchos más requerimientos

In [178]:
#  el número de variables explicativas **nPredictores**
#  un vector **y** de variable a explicar
#  una matriz **x** de variables explicativas, se construye columna a columna usando variables aleatorias
#  calcula la suma de cuadrados totales usando aritmética vectorial
#  esta función debería sustituirse por la lectura de datos desde un fichero
function init_data()
    nObservaciones=25
    nPredictores=15
    x=ones(nObservaciones)
    for i in 1:nPredictores
        x = [x rand(nObservaciones) ]
    end
    y = rand(nObservaciones)
    calculoMedia= Array(Float64,nObservaciones)
    fill!(calculoMedia,mean(y))
    calculoMedia = calculoMedia .- y # suma de cuadrados totales
    nObservaciones,nPredictores,x, y,dot(calculoMedia,calculoMedia)
end

init_data (generic function with 1 method)

In [179]:
# función que selecciona un subconjunto de la matriz X.
# input:
#    matriz: la matriz de variables explicativas
#    nPredictores: el número total de variables explicativas que podemos seleccionar (se usa para recorrer por todas y ver si son activas)
#    seleccion: vector 0/1 que tiene valor igual a 1 si la variable debe estra dentro del modelo
#
# output:
#    ret: matriz con el subconjunto de variables a usar
#
# notas:
#   al menos uno de los coeficientes de selección debe valer 1 para crear una matriz de dos dimensiones, en caso contrario el codigo falla
#   Siempre se considera que hay término independiente en el cálculo, que corresponde a la posición 1 de la matriz (columna de 1's)
#   no es lo más "limpio" pero como proof-of-concept es suficiente
function sub_data(matriz,nPredictores,seleccion)
    count=1
    ret = matriz[:,1]
    for i in 1:nPredictores
        if seleccion[i]==1
            ret=vcat(ret,matriz[:,i+1])
            count += 1
            #ret = [ ret ; matriz[:,i+1] ]
        end
    end
    ret=reshape(ret,size(matriz,1),count)
    ret
end

sub_data (generic function with 1 method)

In [180]:
function determinarR2(nObservaciones,nPredictores,X,y,SStot)
    #println("\nX: ",X)
    #println("\ny: ",y)
    b = X\y
    #println("\nb:",b)
    estimados= X * b
    error= dot(y-estimados,y-estimados)
    1.0-error/SStot
end

determinarR2 (generic function with 1 method)

In [193]:
function main()   #inicio del programa principal
    srand(0) #inicializa la semilla aleatoria para reproducibilidad
    maxNumEnPQ = 500 #valor de control de uso de memoria en el algoritmo
    nObservaciones,nPredictores,X, y,SStot = init_data()
    println("SStot: ",SStot)
    selector = zeros(Int,nPredictores)

    enPQ = Dict()
    numEnPQ = 0
    pqAnterior = Collections.PriorityQueue(Array{Int},Float64)
    Collections.enqueue!(pqAnterior,selector,0.0)
    pqPosterior = Collections.PriorityQueue(Array{Int},Float64)
    etapa = 1
    for etapa in 1:nPredictores
        @printf("etapa: %d  ",etapa)
        println(Collections.length(pqAnterior))
        #se van seleccionando los vertices en pAnterior, se desarrollan y se dejan en pqPosterior
        while Collections.isempty(pqAnterior) == false       
            selector = Collections.dequeue!(pqAnterior)
            for i in 1:nPredictores
                if selector[i] == 0 #si no se había escogido
                    selector[i] = 1 # lo incluyo en la lista
                    retorno=get!(enPQ,selector,0)
                    if(retorno==0)  #no está en la memoria (es nueva) y hay que mirar si se guarda
                        #la guardo en el diccionario de referencia rápida con un 1 que significa que está
                        enPQ[copy(selector)]=1 
                        Xalterado = sub_data(X,nPredictores,selector) #determino r2 (paso 1)
                        r2 = determinarR2(nObservaciones,nPredictores,Xalterado,y,SStot) #determino r2 (paso 2)
                        #println("r2: ",r2)
                        if numEnPQ < maxNumEnPQ  #lo guardo siempre porque faltan alternativas
                            Collections.enqueue!(pqPosterior,deepcopy(selector),r2)
                            numEnPQ += 1
                        else #tengo que ver si es mejor que el peor en PQ
                            descarte, comparacion = Collections.peek(pqPosterior) #leo el valor de la peor
                            if comparacion < r2 #si es mejor, entonces elimino la peor y añado la nueva
                                Collections.dequeue!(pqPosterior)
                                Collections.enqueue!(pqPosterior,deepcopy(selector),r2)
                            end
                        end
                    end
                    selector[i]=0 #para volver a inicio
                end
            end #for
        end #while
      
        while isempty(pqPosterior) == false
            selector,valor = Collections.peek(pqPosterior)
            pr
            Collections.enqueue!(pqAnterior,Collections.dequeue!(pqPosterior),0.0)
        end
        Collections.empty!(enPQ)       # y hay que vaciar la memoria
        numEnPQ = 0
    end
end

main (generic function with 1 method)

In [194]:
main()

SStot: 2.0120061917375214
etapa: 1  1
etapa: 2  15
etapa: 3  105
etapa: 4  455
etapa: 5  500
etapa: 6  500
etapa: 7  500
etapa: 8  500
etapa: 9  500
etapa: 10  500
etapa: 11  500
etapa: 12  424
etapa: 13  226
etapa: 14  73
etapa: 15  13
selector: 1	valor: 1
